In [1]:
import openmm as mm
# import shlex
# import subprocess


# import matplotlib as mpl
# import matplotlib.pyplot as plt

# import pdbfixer
import numpy as np
import openmm.app as app
import simtk.unit as unit
from openmmforcefields.generators import GAFFTemplateGenerator as gen
from openff.toolkit import Topology
import pytraj as pt
import os

from openmm.app import CharmmPsfFile, CharmmCrdFile, CharmmParameterSet
import MDAnalysis as mda

In [2]:
base = '/scratch/htc/fsafarov/structures/8ef5_w_i/charmm-gui/charmm-gui'

#We use psf and crd system because pdb files does not support the 6-digit base system for ATOM ID's, which is in our case utilized for POPC. 
#Input below is from psf and crd from CHARMM-GUI.

psf = CharmmPsfFile(os.path.join(base, 'step5_assembly.psf'))
crd = CharmmCrdFile(os.path.join(base, 'step5_assembly.crd'))

#Input all the parameters from toppar folder of CHARMM-GUI output for all the components of psf and crd.

params = CharmmParameterSet(
    os.path.join(base, 'toppar/top_all36_prot.rtf'),
    os.path.join(base, 'toppar/par_all36m_prot.prm'),
    os.path.join(base, 'toppar/top_all36_na.rtf'),
    os.path.join(base, 'toppar/par_all36_na.prm'),
    os.path.join(base, 'toppar/top_all36_carb.rtf'),
    os.path.join(base, 'toppar/par_all36_carb.prm'),
    os.path.join(base, 'toppar/top_all36_lipid.rtf'),
    os.path.join(base, 'toppar/par_all36_lipid.prm'),
    os.path.join(base, 'toppar/top_all36_cgenff.rtf'),
    os.path.join(base, 'toppar/par_all36_cgenff.prm'),
    os.path.join(base, 'toppar/toppar_all36_moreions.str'),
    os.path.join(base, 'toppar/top_interface.rtf'),
    os.path.join(base, 'toppar/par_interface.prm'),
    os.path.join(base, 'toppar/toppar_all36_nano_lig.str'),
    os.path.join(base, 'toppar/toppar_all36_nano_lig_patch.str'),
    os.path.join(base, 'toppar/toppar_all36_synthetic_polymer.str'),
    os.path.join(base, 'toppar/toppar_all36_synthetic_polymer_patch.str'),
    os.path.join(base, 'toppar/toppar_all36_polymer_solvent.str'),
    os.path.join(base, 'toppar/toppar_water_ions.str'),
    os.path.join(base, 'toppar/toppar_dum_noble_gases.str'),
    os.path.join(base, 'toppar/toppar_ions_won.str'),
    os.path.join(base, 'toppar/cam.str'),
    os.path.join(base, 'toppar/toppar_all36_prot_arg0.str'),
    os.path.join(base, 'toppar/toppar_all36_prot_c36m_d_aminoacids.str'),
    os.path.join(base, 'toppar/toppar_all36_prot_fluoro_alkanes.str'),
    os.path.join(base, 'toppar/toppar_all36_prot_heme.str'),
    os.path.join(base, 'toppar/toppar_all36_prot_na_combined.str'),
    os.path.join(base, 'toppar/toppar_all36_prot_retinol.str'),
    os.path.join(base, 'toppar/toppar_all36_prot_model.str'),
    os.path.join(base, 'toppar/toppar_all36_prot_modify_res.str'),
    os.path.join(base, 'toppar/toppar_all36_na_nad_ppi.str'),
    os.path.join(base, 'toppar/toppar_all36_na_rna_modified.str'),
    os.path.join(base, 'toppar/toppar_all36_lipid_sphingo.str'),
    os.path.join(base, 'toppar/toppar_all36_lipid_archaeal.str'),
    os.path.join(base, 'toppar/toppar_all36_lipid_bacterial.str'),
    os.path.join(base, 'toppar/toppar_all36_lipid_cardiolipin.str'),
    os.path.join(base, 'toppar/toppar_all36_lipid_cholesterol.str'),
    os.path.join(base, 'toppar/toppar_all36_lipid_dag.str'),
    os.path.join(base, 'toppar/toppar_all36_lipid_inositol.str'),
    os.path.join(base, 'toppar/toppar_all36_lipid_lnp.str'),
    os.path.join(base, 'toppar/toppar_all36_lipid_lps.str'),
    os.path.join(base, 'toppar/toppar_all36_lipid_mycobacterial.str'),
    os.path.join(base, 'toppar/toppar_all36_lipid_miscellaneous.str'),
    os.path.join(base, 'toppar/toppar_all36_lipid_model.str'),
    os.path.join(base, 'toppar/toppar_all36_lipid_prot.str'),
    os.path.join(base, 'toppar/toppar_all36_lipid_tag.str'),
    os.path.join(base, 'toppar/toppar_all36_lipid_yeast.str'),
    os.path.join(base, 'toppar/toppar_all36_lipid_hmmm.str'),
    os.path.join(base, 'toppar/toppar_all36_lipid_detergent.str'),
    os.path.join(base, 'toppar/toppar_all36_lipid_ether.str'),
    os.path.join(base, 'toppar/toppar_all36_lipid_oxidized.str'),
    os.path.join(base, 'toppar/toppar_all36_carb_glycolipid.str'),
    os.path.join(base, 'toppar/toppar_all36_carb_glycopeptide.str'),
    os.path.join(base, 'toppar/toppar_all36_carb_imlab.str'),
    os.path.join(base, 'toppar/toppar_all36_label_spin.str'),
    os.path.join(base, 'toppar/toppar_all36_label_fluorophore.str'),
    os.path.join(base, '7v7/7v7.rtf'),
    os.path.join(base, '7v7/7v7.prm')
)

/scratch/htc/fsafarov/openmm_ff/lib/python3.12/site-packages/openmm/app/charmmparameterset.py:391: UserWarning: Replacing dihedral <DihedralType: k=0.27; phase=180.0; per=2> with <DihedralType: k=0.03; phase=180.0; per=2>
  warnings.warn('Replacing dihedral %r with %r' %
/scratch/htc/fsafarov/openmm_ff/lib/python3.12/site-packages/openmm/app/charmmparameterset.py:391: UserWarning: Replacing dihedral <DihedralType: k=0.0; phase=0.0; per=3> with <DihedralType: k=0.12; phase=0.0; per=3>
  warnings.warn('Replacing dihedral %r with %r' %


In [3]:
psf.setBox(100.167755*unit.angstroms, 100.167755*unit.angstroms, 127.064628*unit.angstrom)
#Set up a box

In [4]:
n1_atom = psf.topology.getNumAtoms()
n1_atom

118459

In [5]:
print(psf.topology.getPeriodicBoxVectors())
#check if the pbc is working

(Vec3(x=10.016775500000001, y=0.0, z=0.0), Vec3(x=0.0, y=10.016775500000001, z=0.0), Vec3(x=0.0, y=0.0, z=12.7064628)) nm


In [6]:
# fixer = pdbfixer.PDBFixer(filename='system_box.pdb')
# fixer.findMissingResidues()
# fixer.findMissingAtoms()
# fixer.addMissingAtoms()
# fixer.addMissingHydrogens()

In [7]:
print(params.atom_types_int);
#Verifying if all the atoms are parametrized

{-1: <openmm.app.internal.charmm.topologyobjects.AtomType object at 0x7ff69f2ae870>}


In [8]:
system = psf.createSystem(params, nonbondedMethod=app.PME, nonbondedCutoff=1.0 * unit.nanometer, constraints = app.HBonds)
#constraints = app.HBonds::We constraint the Hydrogen bonds so that they won\t stretch during the simulation

In [9]:
restraint = mm.CustomExternalForce('k*periodicdistance(x, y, z, x0, y0, z0)^2')
# system.addForce(restraint)non-standard (HET) residues (excluding water).
# - Inter-residue connectivity of HET  groups to standard groups (including water) or to other HET groups. 
# - Disulfide bridges specified in the  SSBOND records have corresponding records.

restraint.addGlobalParameter('k', 35.0*unit.kilojoules_per_mole/unit.nanometer**2)
restraint.addPerParticleParameter('x0')
restraint.addPerParticleParameter('y0')
restraint.addPerParticleParameter('z0')
system.addForce(restraint)

9

In [10]:
# Centering the solute within the periodic box before running the simulation
# This step is not strictly required for the simulation to run correctly,
# but without it, the periodic box may appear misaligned with the structure,
# causing the protein (or membrane) to drift outside the visible box in trajectory files.
# Centering improves visualization and helps ensure proper PBC wrapping in trajectory output.
positions_check = crd.positions
center = np.mean(positions_check.value_in_unit(unit.nanometer), axis=0)
box = psf.topology.getUnitCellDimensions().value_in_unit(unit.nanometer)
box_center = np.array(box) / 2.0
translation = box_center - center
centered_positions = positions_check + translation * unit.nanometer
centered_positions = centered_positions.value_in_unit(unit.nanometer)

In [11]:
for residue in psf.topology.residues():
    if residue.name == 'POPC':
        for atom in residue.atoms():
            pos = centered_positions[atom.index]
            restraint.addParticle(atom.index, [pos[0], pos[1], pos[2]])

In [12]:
integrator_eq = mm.LangevinIntegrator(310.10*unit.kelvin, 1/unit.picoseconds, 2.0*unit.femtoseconds)

In [13]:
system.addForce(mm.MonteCarloBarostat(1*unit.atmosphere, 310.10*unit.kelvin))

10

In [14]:
platform = mm.Platform.getPlatformByName('CUDA')

In [15]:
#loads checkpoint if exists
simulation_eq = app.Simulation(psf.topology, system, integrator_eq, platform)
if os.path.exists('/scratch/htc/fsafarov/traj/checkpoint3.chk'):
    simulation_eq.loadCheckpoint('/scratch/htc/fsafarov/traj/checkpoint3.chk')
else:
    simulation_eq.context.setPositions(centered_positions)
    box_vectors = psf.topology.getPeriodicBoxVectors()
    simulation_eq.context.setPeriodicBoxVectors(*box_vectors)

In [16]:
simulation_eq.context.getPlatform().getName()

'CUDA'

In [17]:
for i in range(5):
    print(crd.positions[i])

Vec3(x=37.6858657556, y=0.9888895581, z=-18.8703364334) A
Vec3(x=38.6367620965, y=0.6241948639, z=-18.5827712699) A
Vec3(x=37.07998828, y=0.8088928581, z=-18.0251067183) A
Vec3(x=37.7901802562, y=1.9974508636, z=-19.0265588286) A
Vec3(x=37.1534179002, y=0.2329982214, z=-20.0699441249) A


In [18]:
# #Check position units, if there is any mismatch. NO MISMATCH
# #positions_check = simulation_eq.context.getState(getPositions=True).getPositions()

# for i in range(5):
#     print(positions_check[i])



In [19]:
# Before minimization
state = simulation_eq.context.getState(getEnergy=True)
print("Potential energy before minimization:", state.getPotentialEnergy())

simulation_eq.minimizeEnergy()
#default number of iterations is unlimited. maxiterations = 0 mean unlimited.

# After minimization
state = simulation_eq.context.getState(getEnergy=True)
print("Potential energy after minimization:", state.getPotentialEnergy())


Potential energy before minimization: -1087470.385296912 kJ/mol
Potential energy after minimization: -1487030.705609412 kJ/mol


In [20]:
# steps_per_microsecond = int(1e6 / (2.4 * 1e-3))# Number of steps per µs with 2.4 fs timestep
# print(steps_per_microsecond)
simulation_eq.reporters.append(app.StateDataReporter('/scratch/htc/fsafarov/traj/output__w_rest_long11.log', 10000, step=True, potentialEnergy=True, temperature=True))
simulation_eq.reporters.append(app.DCDReporter('/scratch/htc/fsafarov/traj/trajectory_w_rest_long11.dcd', 10000, enforcePeriodicBox=True))
simulation_eq.reporters.append(app.CheckpointReporter('/scratch/htc/fsafarov/traj/checkpoint3.chk', 100000))



In [21]:
simulation_eq.currentStep

32500000

In [ ]:
#progress the simulation in chunks (too much frames at ones for longer simulation)
completed_steps = simulation_eq.currentStep
while completed_steps < 50000000:
    simulation_eq.step(5000000)
    completed_steps += 5000000
    simulation_eq.saveCheckpoint('/scratch/htc/fsafarov/traj/checkpoint3.chk')
    print(f"{completed_steps} steps are done and written into the checkpoint file.")

In [ ]:

# pos = simulation_eq.context.getState(getPositions=True).getPositions()

# with open("final_output.pdb", "w") as file_:
#     app.PDBFile.writeFile(simulation_eq.topology, pos, file_)


In [ ]:
# for atom in psf.topology.residues():
#     print(atom.name)